## Projeto PBD

In [2]:
import os
import sys

import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
# from ydata_profiling import ProfileReport
import warnings
warnings.filterwarnings("ignore")

In [1]:
import sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

Intitializing PySpark ...

23/04/04 19:43:32 WARN Utils: Your hostname, DESKTOP-M9FP7G5 resolves to a loopback address: 127.0.1.1; using 192.168.89.130 instead (on interface eth0)
23/04/04 19:43:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/04/04 19:43:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Web UI available at http://192.168.89.130:4040
SparkContext available as 'sc' (version = 3.3.1, master = local[*], app id = local-1680633816670)
SparkSession available as 'spark'


In [3]:
from pyspark.sql import SparkSession

# build our own SparkSession
spark = SparkSession\
    .builder\
    .appName("BigData")\
    .config("spark.sql.shuffle.partitions",6)\
    .config("spark.sql.repl.eagereval.enabled",True)\
    .getOrCreate() 

23/04/04 19:43:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
spark

### 1ªInicio
* Read CSV
* Perform EDA

	- Understand the problem
	- Which one makes sense to keep for Ranking

In [5]:
! head -n 2 train.csv
! tail -n 1 train.csv

date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,9,2014-08-27,2014-08-31,2,0,1,8250,1,0,3,2,50,628,1

2014-09-18 08:52:42,2,3,66,462,12565,106.4274,1198182,0,0,0,2014-09-18,2014-09-19,1,0,1,18811,1,1,1,2,50,592,42



In [6]:
filename = 'train.csv'
df_hotels =  spark.read.csv(filename, header=True, inferSchema="true", sep=',')

### Check DATA

In [7]:
df_hotels.printSchema()
df_hotels.show(1, vertical=True)
hotels_count = df_hotels.count()
hotels_count

root
 |-- date_time: timestamp (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- user_location_region: integer (nullable = true)
 |-- user_location_city: integer (nullable = true)
 |-- orig_destination_distance: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- is_mobile: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- channel: integer (nullable = true)
 |-- srch_ci: timestamp (nullable = true)
 |-- srch_co: timestamp (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_rm_cnt: integer (nullable = true)
 |-- srch_destination_id: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- is_booking: integer (nullable = true)
 |-- cnt: integer (nullable = true)
 |-- hotel_continent: integer (nullable = true)
 |-- hotel_country: 

37670293

### **Evaluate DATA**


### **Drop column user_id**

In [8]:
df_hotels = df_hotels.drop('user_id')

In [9]:
df_hotels.show(1, vertical=True)

-RECORD 0----------------------------------------
 date_time                 | 2014-08-11 07:46:59 
 site_name                 | 2                   
 posa_continent            | 3                   
 user_location_country     | 66                  
 user_location_region      | 348                 
 user_location_city        | 48862               
 orig_destination_distance | 2234.2641           
 is_mobile                 | 0                   
 is_package                | 1                   
 channel                   | 9                   
 srch_ci                   | 2014-08-27 00:00:00 
 srch_co                   | 2014-08-31 00:00:00 
 srch_adults_cnt           | 2                   
 srch_children_cnt         | 0                   
 srch_rm_cnt               | 1                   
 srch_destination_id       | 8250                
 srch_destination_type_id  | 1                   
 is_booking                | 0                   
 cnt                       | 3                   


### Criar nº de noites e ID_hotel

In [12]:
df_hotels = df_hotels.withColumn('Id_hotel', 
                    F.concat(F.col('hotel_cluster'),F.lit('_'), F.col('hotel_market'),F.lit('_'), F.col('hotel_country'),F.lit('_'), F.col('hotel_continent')))
df_hotels.show(5,vertical = True)


-RECORD 0----------------------------------------
 date_time                 | 2014-08-11 07:46:59 
 site_name                 | 2                   
 posa_continent            | 3                   
 user_location_country     | 66                  
 user_location_region      | 348                 
 user_location_city        | 48862               
 orig_destination_distance | 2234.2641           
 is_mobile                 | 0                   
 is_package                | 1                   
 channel                   | 9                   
 srch_ci                   | 2014-08-27 00:00:00 
 srch_co                   | 2014-08-31 00:00:00 
 srch_adults_cnt           | 2                   
 srch_children_cnt         | 0                   
 srch_rm_cnt               | 1                   
 srch_destination_id       | 8250                
 srch_destination_type_id  | 1                   
 is_booking                | 0                   
 cnt                       | 3                   


In [51]:
df_hotels=df_hotels.withColumn("n_of_nights", datediff(F.col("srch_co"),F.col("srch_ci"))-1)

In [52]:
df_hotels.show(3,vertical=True)

-RECORD 0---------------------------------------
 date_time                | 2014-08-11 07:46:59 
 site_name                | 2                   
 posa_continent           | 3                   
 user_location_country    | 66                  
 user_location_region     | 348                 
 user_location_city       | 48862               
 is_mobile                | 0                   
 is_package               | 1                   
 channel                  | 9                   
 srch_ci                  | 2014-08-27 00:00:00 
 srch_co                  | 2014-08-31 00:00:00 
 srch_adults_cnt          | 2                   
 srch_children_cnt        | 0                   
 srch_rm_cnt              | 1                   
 srch_destination_id      | 8250                
 srch_destination_type_id | 1                   
 is_booking               | 0                   
 cnt                      | 3                   
 hotel_continent          | 2                   
 hotel_country      

### **NULLS**

In [13]:
print('\nNulls in Hotels:')
cols_to_forget = ['date_time','srch_ci','srch_co']
hotels_cols_interest = [x for x in df_hotels.columns if x not in cols_to_forget]
for cl in hotels_cols_interest:
    k = df_hotels.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {hotels_count} records ({k*100/hotels_count:.2f}%)')


Nulls in Hotels:
Column orig_destination_distance with 13525001 nulls or NaN, out of 37670293 records (35.90%)


In [14]:
hotels_cols_interest.remove('orig_destination_distance')
df_hotels = df_hotels.drop('orig_destination_distance')

In [15]:
df_hotels.show(1, vertical=True)

-RECORD 0---------------------------------------
 date_time                | 2014-08-11 07:46:59 
 site_name                | 2                   
 posa_continent           | 3                   
 user_location_country    | 66                  
 user_location_region     | 348                 
 user_location_city       | 48862               
 is_mobile                | 0                   
 is_package               | 1                   
 channel                  | 9                   
 srch_ci                  | 2014-08-27 00:00:00 
 srch_co                  | 2014-08-31 00:00:00 
 srch_adults_cnt          | 2                   
 srch_children_cnt        | 0                   
 srch_rm_cnt              | 1                   
 srch_destination_id      | 8250                
 srch_destination_type_id | 1                   
 is_booking               | 0                   
 cnt                      | 3                   
 hotel_continent          | 2                   
 hotel_country      

### **Summary to figure out outliers**

In [53]:
df_hotels.describe(hotels_cols_interest).show(vertical = True)

-RECORD 0---------------------------------------
 summary                  | count               
 site_name                | 37670293            
 posa_continent           | 37670293            
 user_location_country    | 37670293            
 user_location_region     | 37670293            
 user_location_city       | 37670293            
 user_id                  | 37670293            
 is_mobile                | 37670293            
 is_package               | 37670293            
 channel                  | 37670293            
 srch_adults_cnt          | 37670293            
 srch_children_cnt        | 37670293            
 srch_rm_cnt              | 37670293            
 srch_destination_id      | 37670293            
 srch_destination_type_id | 37670293            
 is_booking               | 37670293            
 cnt                      | 37670293            
 hotel_continent          | 37670293            
 hotel_country            | 37670293            
 hotel_market       

In [54]:
df_hotels.select('hotel_cluster').distinct().count()

100

In [55]:
df_hotels.select('Id_hotel').distinct().count()

55694

### **UNIQUENESS**

In [15]:
print('\nUniqueness in Hotels:')
for cl in hotels_cols_interest:
    k = df_hotels.select(cl).distinct().count()
    if k > 0:
        print(f'Column {cl} with {k} distinct values, out of {hotels_count} records ({k*100/hotels_count:.2f}%)')


Uniqueness in Hotels:
Column site_name with 45 distinct values, out of 37670293 records (0.00%)
Column posa_continent with 5 distinct values, out of 37670293 records (0.00%)
Column user_location_country with 237 distinct values, out of 37670293 records (0.00%)
Column user_location_region with 1008 distinct values, out of 37670293 records (0.00%)
Column user_location_city with 50447 distinct values, out of 37670293 records (0.13%)
Column orig_destination_distance with 8495290 distinct values, out of 37670293 records (22.55%)
Column is_mobile with 2 distinct values, out of 37670293 records (0.00%)
Column is_package with 2 distinct values, out of 37670293 records (0.00%)
Column channel with 11 distinct values, out of 37670293 records (0.00%)
Column srch_adults_cnt with 10 distinct values, out of 37670293 records (0.00%)
Column srch_children_cnt with 10 distinct values, out of 37670293 records (0.00%)
Column srch_rm_cnt with 9 distinct values, out of 37670293 records (0.00%)
Column srch_d

# **Saving clean data**


In [ ]:
seed = 5
with_replacement = False
fraction = 0.01          
small_df_hotels = df_hotels.sample(withReplacement=with_replacement, 
                                               fraction=fraction, seed=seed)
small_df_hotels.count()

In [ ]:
small_df_hotels.write.mode("overwrite").parquet("small-hotels")

### **Final Overview**

In [ ]:
df_clean = small_df_hotels

In [ ]:
hotels_count = df_clean.count()
hotels_count

### **Train Model**

In [37]:
df_clean = spark.read.parquet('small-hotels')

In [38]:
# Importando as bibliotecas necessárias
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.recommendation import ALS

# Criando um vetor com os atributos que serão usados para fazer as recomendações
vectorAssembler = VectorAssembler(inputCols=["srch_children_cnt", "is_mobile", "is_booking", "is_package", "user_location_country"], outputCol="features")
df_clean = vectorAssembler.transform(df_clean)

# Separando os dados em conjuntos de treinamento e teste
(training, test) = df_clean.randomSplit([0.8, 0.2])

# Configurando o modelo ALS (Alternating Least Squares) para fazer as recomendações
als = ALS(userCol="user_id", itemCol="hotel_cluster", ratingCol="is_booking", coldStartStrategy="drop")
model = als.fit(training)

# Fazendo as recomendações para cada usuário com base em seus históricos de reservas e outros atributos
recommendations = model.recommendForAllUsers(10)

# Mostrando as recomendações para um usuário específico
recommendations.filter(recommendations.user_id == 1234).show()

23/03/23 14:20:22 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
+-------+---------------+
|user_id|recommendations|
+-------+---------------+
+-------+---------------+



In [21]:
recommendations.filter(recommendations.user_id == 1234).show()

NameError: name 'recommendations' is not defined

In [ ]:
df_hotels.groupBy('posa_continent').count().sort(F.desc('count')).show()

In [ ]:
df_hotels.groupBy('site_name').count().sort(F.desc('count')).show()